# Action Recognition @ UCF101  
**Due date: 11:59 pm on Nov. 19, 2019 (Tuesday)**

## Description
---
In this homework, you will be doing action recognition using Recurrent Neural Network (RNN), (Long-Short Term Memory) LSTM in particular. You will be given a dataset called UCF101, which consists of 101 different actions/classes and for each action, there will be 145 samples. We tagged each sample into either training or testing. Each sample is supposed to be a short video, but we sampled 25 frames from each videos to reduce the amount of data. Consequently, a training sample is an image tuple that forms a 3D volume with one dimension encoding *temporal correlation* between frames and a label indicating what action it is.

To tackle this problem, we aim to build a neural network that can not only capture spatial information of each frame but also temporal information between frames. Fortunately, you don't have to do this on your own. RNN — a type of neural network designed to deal with time-series data — is right here for you to use. In particular, you will be using LSTM for this task.

Instead of training an end-to-end neural network from scratch whose computation is prohibitively expensive, we divide this into two steps: feature extraction and modelling. Below are the things you need to implement for this homework:
- **{35 pts} Feature extraction**. Use any of the [pre-trained models](https://pytorch.org/docs/stable/torchvision/models.html) to extract features from each frame. Specifically, we recommend not to use the activations of the last layer as the features tend to be task specific towards the end of the network. 
    **hints**: 
    - A good starting point would be to use a pre-trained VGG16 network, we suggest first fully connected layer `torchvision.models.vgg16` (4096 dim) as features of each video frame. This will result into a 4096x25 matrix for each video. 
    - Normalize your images using `torchvision.transforms` 
    ```
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    prep = transforms.Compose([ transforms.ToTensor(), normalize ])
    prep(img)
    The mean and std. mentioned above is specific to Imagenet data
    
    ```
    More details of image preprocessing in PyTorch can be found at http://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    
- **{35 pts} Modelling**. With the extracted features, build an LSTM network which takes a **dx25** sample as input (where **d** is the dimension of the extracted feature for each frame), and outputs the action label of that sample.
- **{20 pts} Evaluation**. After training your network, you need to evaluate your model with the testing data by computing the prediction accuracy **(5 points)**. The baseline test accuracy for this data is 75%, and **10 points** out of 20 is for achieving test accuracy greater than the baseline. Moreover, you need to compare **(5 points)** the result of your network with that of support vector machine (SVM) (stacking the **dx25** feature matrix to a long vector and train a SVM).
- **{10 pts} Report**. Details regarding the report can be found in the submission section below.

Notice that the size of the raw images is 256x340, whereas your pre-trained model might take **nxn** images as inputs. To solve this problem, instead of resizing the images which unfavorably changes the spatial ratio, we take a better solution: Cropping five **nxn** images, one at the image center and four at the corners and compute the **d**-dim features for each of them, and average these five **d**-dim feature to get a final feature representation for the raw image.
For example, VGG takes 224x224 images as inputs, so we take the five 224x224 croppings of the image, compute 4096-dim VGG features for each of them, and then take the mean of these five 4096-dim vectors to be the representation of the image.

In order to save you computational time, you need to do the classification task only for **the first 25** classes of the whole dataset. The same applies to those who have access to GPUs. **Bonus 10 points for running and reporting on the entire 101 classes.**


## Dataset
Download **dataset** at [UCF101](http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_images.tar)(Image data for each video) and the **annos folder** which has the video labels and the label to class name mapping is included in the assignment folder uploaded. 


UCF101 dataset contains 101 actions and 13,320 videos in total.  

+ `annos/actions.txt`  
  + lists all the actions (`ApplyEyeMakeup`, .., `YoYo`)   
  
+ `annots/videos_labels_subsets.txt`  
  + lists all the videos (`v_000001`, .., `v_013320`)  
  + labels (`1`, .., `101`)  
  + subsets (`1` for train, `2` for test)  

+ `images/`  
  + each folder represents a video
  + the video/folder name to class mapping can be found using `annots/videos_labels_subsets.txt`, for e.g. `v_000001` belongs to class 1 i.e. `ApplyEyeMakeup`
  + each video folder contains 25 frames  



## Some Tutorials
- Good materials for understanding RNN and LSTM
    - http://blog.echen.me
    - http://karpathy.github.io/2015/05/21/rnn-effectiveness/
    - http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Implementing RNN and LSTM with PyTorch
    - [LSTM with PyTorch](http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py)
    - [RNN with PyTorch](http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

## Preparation

In [0]:
import numpy as np
import os
import gc
import shutil
import torch
import torchvision
import torchvision.models as models
from torch.autograd import Variable 
import torch.cuda
import torchvision.transforms as transforms
from PIL import Image
import pickle
from torch import nn
import torch.nn.functional as F
from sklearn.svm import LinearSVC

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
os.chdir('/content/gdrive/My Drive/Ma_Yuchen_112877697_hw5')
os.listdir()

['test', 'name.txt', 'map.txt', 'train', 'Ma_Yuchen_112877697_hw5.ipynb']

In [0]:
  content = []
  with open('name.txt', 'r') as f:
      for line in f.readlines():
          content.append([line.strip('\n')])

  class_name = []
  for i in range(0, 101):
    t = content[i][0]
    s = ''
    for j in range(5, len(t)):
      s = s + content[i][0][j]
    class_name.append([s])

  content1 = []
  with open('map.txt', 'r') as f:
      for line in f.readlines():
          content1.append([line.strip('\n')])

  file_name = []
  for i in range(0, 13320):
    t = content1[i][0]
    s = ''
    for j in range(0,8):
      s = s + content1[i][0][j]
    file_name.append([s])

  file_class_corr_num = []
  for i in range(0, 13320):
    t = content1[i][0]
    s = ''
    for j in range(9,len(t)-2):
      s = s + content1[i][0][j]
    int(s)
    file_class_corr_num.append([s])

  label_ = []
  for i in range(0, 13320):
    t = content1[i][0]
    s = content1[i][0][len(t)-1]
    label_.append([s])

  # for i in range(0,13320):
  # if label_[i][0] == '1':
  #   src = '/content/gdrive/My Drive/Ma_Yuchen_112877697_hw5/images/' + file_name[i][0]
  #   dst = '/content/gdrive/My Drive/Ma_Yuchen_112877697_hw5/train/' + name[int(file_num[i][0])-1][0]
  #   shutil.move(src,dst)
  # if label_[i][0] == '2':
  #   src = '/content/gdrive/My Drive/Ma_Yuchen_112877697_hw5/images/' + file_name[i][0]
  #   dst = '/content/gdrive/My Drive/Ma_Yuchen_112877697_hw5/test/' + name[int(file_num[i][0])-1][0]
  #   shutil.move(src,dst)

---
---
## **Problem 1.** Feature extraction

In [0]:
model = models.vgg16(pretrained=True)
model.classifier = model.classifier[:1]
model = model.eval()
feature_size = 224

def extract_feature(img):
    norm = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    feature_transform = transforms.Compose([   
        transforms.FiveCrop(feature_size),
        transforms.Lambda(lambda crops: torch.stack([norm(transforms.ToTensor()(crop)) for crop in crops]))
    ])
    img_data = feature_transform(img)
    with torch.no_grad():
        feature = torch.mean(model(img_data),dim=0)
    return feature

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:13<00:00, 39.8MB/s]


In [0]:
for i in range(0,25):
  src = '/content/gdrive/My Drive/Ma_Yuchen_112877697_hw5/train/' + class_name[i][0]
  folder = torchvision.datasets.ImageFolder(src)
  feature_list = []
  flag = 0
  for c, images in enumerate(folder):
    feature_list += [extract_feature(images[0])]
    flag = flag+1
    if (flag%500==0):
      print(flag)

  list_ = []
  k = 0
  while (k < len(feature_list)):
    temp = torch.stack([feature_list[k+j] for j in range(25)])
    list_ += [temp]
    k = k + 25

  import pickle
  output = open(str(i+1)+'.pkl', 'wb')
  pickle.dump(list_, output)
  output.close()
  print(str(i+1)+'.pkl')
  print('***********************************')

In [0]:
temp = []
for i in range(1,26):
  pkl_file = open(str(i)+'.pkl', 'rb')
  data = pickle.load(pkl_file)
  temp.append(data)
  print(str(i)+'.pkl')
  pkl_file.close()

In [0]:
output = open('test.pkl', 'wb')
pickle.dump(temp,output)
output.close()

***
***
## **Problem 2.** Modelling

* ##### **Print the size of your training and test data**

In [0]:
# Don't hardcode the shape of train and test data

os.chdir('/content/gdrive/My Drive/Ma_Yuchen_112877697_hw5/train')
file1 = open('train.pkl', 'rb')
train_data = pickle.load(file1)
train = []
train_label = []
for i in range(0,25):
  for j in range(0, len(train_data[i])):
    train.append(train_data[i][j])
    train_label.append(torch.LongTensor(1).fill_(i))
train = torch.stack(train, 0)
train_label = torch.stack(train_label, 0)

os.chdir('/content/gdrive/My Drive/Ma_Yuchen_112877697_hw5/test')
file2 = open('test.pkl', 'rb')
test_data = pickle.load(file2)
test = []
test_label = []
for i in range(0,25):
  for j in range(0, len(test_data[i])):
    test.append(test_data[i][j])
    test_label.append(torch.LongTensor(1).fill_(i))
test = torch.stack(test, 0)
test_label = torch.stack(test_label, 0)

print('Shape of training data is :', train.shape)
print('Shape of test/validation data is :', test.shape)

Shape of training data is : torch.Size([2409, 25, 4096])
Shape of test/validation data is : torch.Size([951, 25, 4096])


In [0]:
class lstm(nn.Module):
    def __init__(self):
        super(lstm, self).__init__()
        self.hidden_dim = 256
        self.lstm = nn.LSTM(
            input_size = 4096,
            hidden_size = 256,
            num_layers = 1,
            batch_first = True)
        self.out = nn.Linear(256, 25)

    def forward(self, img):
        lstm_out, _ = self.lstm(img, None)
        tag_out = self.out(lstm_out[:,-1,:])
        tag_scores = F.softmax(tag_out, dim=1)
        return tag_scores

model = lstm()
model1 = lstm()
model2 = lstm()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
optimizer1 = torch.optim.SGD(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.SGD(model2.parameters(), lr=0.001)
model

lstm(
  (lstm): LSTM(4096, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=25, bias=True)
)

In [0]:
for epoch in range(100):
  for i in range(0, len(train)):
    model.zero_grad()
    inputs = Variable(train[i].view(1,25,4096))
    targets = train_label[i]
    tag_scores = model(inputs)
    loss = loss_function(tag_scores, targets)
    loss.backward()
    optimizer.step()
print("Done!")

In [0]:
for epoch in range(20):
  for i in range(0, len(train)):
    model1.zero_grad()
    inputs = Variable(train[i].view(1,25,4096))
    targets = train_label[i]
    tag_scores = model1(inputs)
    loss = loss_function(tag_scores, targets)
    loss.backward()
    optimizer1.step()
print("Done!")

Done!


In [0]:
for epoch in range(50):
  for i in range(0, len(train)):
    model2.zero_grad()
    inputs = Variable(train[i].view(1,25,4096))
    targets = train_label[i]
    tag_scores = model2(inputs)
    loss = loss_function(tag_scores, targets)
    loss.backward()
    optimizer2.step()
print("Done!")

Done!


In [0]:
class VariationalDropout(nn.Module):
    def __init__(self, dropout: float, batch_first: bool=False):
        super().__init__()
        self.dropout = dropout
        self.batch_first = batch_first

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if not self.training or self.dropout <= 0.:
            return x

        is_packed = isinstance(x, PackedSequence)
        if is_packed:
            x, batch_sizes = x
            max_batch_size = int(batch_sizes[0])
        else:
            batch_sizes = None
            max_batch_size = x.size(0)

        # Drop same mask across entire sequence
        if self.batch_first:
            m = x.new_empty(max_batch_size, 1, x.size(2), requires_grad=False).bernoulli_(1 - self.dropout)
        else:
            m = x.new_empty(1, max_batch_size, x.size(2), requires_grad=False).bernoulli_(1 - self.dropout)
        x = x.masked_fill(m == 0, 0) / (1 - self.dropout)

        if is_packed:
            return PackedSequence(x, batch_sizes)
        else:
            return x

class better_lstm(nn.LSTM):
    def __init__(self, *args, dropouti: float=0.,
                 dropoutw: float=0., dropouto: float=0.,
                 batch_first=True, unit_forget_bias=True, **kwargs):
        super().__init__(*args, **kwargs, batch_first=batch_first)
        self.unit_forget_bias = unit_forget_bias
        self.dropoutw = dropoutw
        self.input_drop = VariationalDropout(dropouti,
                                             batch_first=batch_first)
        self.output_drop = VariationalDropout(dropouto,
                                              batch_first=batch_first)
        self._init_weights()
        self.out = nn.Linear(256, 25)

    def _init_weights(self):
        for name, param in self.named_parameters():
            if "weight_hh" in name:
                nn.init.orthogonal_(param.data)
            elif "weight_ih" in name:
                nn.init.xavier_uniform_(param.data)
            elif "bias" in name and self.unit_forget_bias:
                nn.init.zeros_(param.data)
                param.data[self.hidden_size:2 * self.hidden_size] = 1

    def _drop_weights(self):
        for name, param in self.named_parameters():
            if "weight_hh" in name:
                getattr(self, name).data = \
                    torch.nn.functional.dropout(param.data, p=self.dropoutw,
                                                training=self.training).contiguous()

    def forward(self, input, hx=None):
        self._drop_weights()
        input = self.input_drop(input)
        seq, state = super().forward(input, hx=hx)
        tag_out = self.out(seq[:,-1,:])
        tag_scores = F.softmax(tag_out, dim=1)
        return tag_scores


better_lstm(
  4096, 256, batch_first=True
  (input_drop): VariationalDropout()
  (output_drop): VariationalDropout()
  (out): Linear(in_features=256, out_features=25, bias=True)
)

In [0]:
class double_lstm(nn.Module):
    def __init__(self):
        super(double_lstm, self).__init__()
        self.hidden_dim = 256
        self.double_lstm= nn.LSTM(
            input_size = 4096,
            hidden_size = 256,
            num_layers = 2,
            batch_first = True)
        self.out = nn.Linear(256, 25)

    def forward(self, img):
        lstm_out, _ = self.double_lstm(img, None)
        tag_out = self.out(lstm_out[:,-1,:])
        tag_scores = F.softmax(tag_out, dim=1)
        return tag_scores

model3 = double_lstm()
model4 = double_lstm()
model5 = double_lstm()
loss_function = nn.CrossEntropyLoss()
optimizer3 = torch.optim.SGD(model3.parameters(), lr=0.001)
optimizer4 = torch.optim.SGD(model4.parameters(), lr=0.001)
optimizer5 = torch.optim.SGD(model5.parameters(), lr=0.0001)
model3

double_lstm(
  (double_lstm): LSTM(4096, 256, num_layers=2, batch_first=True)
  (out): Linear(in_features=256, out_features=25, bias=True)
)

In [0]:
for epoch in range(20):
  print(epoch)
  for i in range(0, len(train)):
    model3.zero_grad()
    inputs = Variable(train[i].view(1,25,4096))
    targets = train_label[i]
    tag_scores = model3(inputs)
    loss = loss_function(tag_scores, targets)
    loss.backward()
    optimizer3.step()
print("Done!")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Done!


In [0]:
for epoch in range(50):
  print(epoch)
  for i in range(0, len(train)):
    model4.zero_grad()
    inputs = Variable(train[i].view(1,25,4096))
    targets = train_label[i]
    tag_scores = model4(inputs)
    loss = loss_function(tag_scores, targets)
    loss.backward()
    optimizer4.step()
print("Done!")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Done!


In [9]:
for epoch in range(100):
  print(epoch)
  for i in range(0, len(train)):
    model5.zero_grad()
    inputs = Variable(train[i].view(1,25,4096))
    targets = train_label[i]
    tag_scores = model5(inputs)
    loss = loss_function(tag_scores, targets)
    loss.backward()
    optimizer5.step()
print("Done!")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Done!


---
---
## **Problem 3.** Evaluation

In [0]:
# \*write your codes for evaluation (You can use multiple cells, this is just a place holder)

* ##### **Print the train and test accuracy of your model** 

In [0]:
# Don't hardcode the train and test accuracy
#epoch = 100
total = 0
correct = 0
with torch.no_grad():
  for i in range(0,len(train)):
   inputs = Variable(train[i].view(1,25,4096))
   targets = train_label[i]
   tag_scores = model(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total += 1
   correct += (pred == targets).sum().item()
print('Training accuracy is :',correct/total*100)

total1 = 0
correct1 = 0
with torch.no_grad():
  for i in range(0,len(test)):
   inputs = Variable(test[i].view(1,25,4096))
   targets = test_label[i]
   tag_scores = model(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total1 += 1
   correct1 += (pred == targets).sum().item()
print('Testing accuracy is :',correct1/total1*100)

In [0]:
# epoch = 20
total2 = 0
correct2 = 0
with torch.no_grad():
  for i in range(0,len(train)):
   inputs = Variable(train[i].view(1,25,4096))
   targets = train_label[i]
   tag_scores = model1(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total2 += 1
   correct2 += (pred == targets).sum().item()
print('Training accuracy is :',correct2/total2*100)

total3 = 0
correct3 = 0
with torch.no_grad():
  for i in range(0,len(test)):
   inputs = Variable(test[i].view(1,25,4096))
   targets = test_label[i]
   tag_scores = model1(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total3 += 1
   correct3 += (pred == targets).sum().item()
print('Testing accuracy is :',correct3/total3*100)

Training accuracy is : 70.1120797011208
Testing accuracy is : 60.042060988433235


In [0]:
# epoch = 50
total4 = 0
correct4 = 0
with torch.no_grad():
  for i in range(0,len(train)):
   inputs = Variable(train[i].view(1,25,4096))
   targets = train_label[i]
   tag_scores = model2(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total4 += 1
   correct4 += (pred == targets).sum().item()
print('Training accuracy is :',correct4/total4*100)

total5 = 0
correct5 = 0
with torch.no_grad():
  for i in range(0,len(test)):
   inputs = Variable(test[i].view(1,25,4096))
   targets = test_label[i]
   tag_scores = model2(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total5 += 1
   correct5 += (pred == targets).sum().item()
print('Testing accuracy is :',correct5/total5*100)

Training accuracy is : 91.65628891656289
Testing accuracy is : 75.49947423764458


In [0]:
# epoch = 20
total6 = 0
correct6 = 0
with torch.no_grad():
  for i in range(0,len(train)):
   inputs = Variable(train[i].view(1,25,4096))
   targets = train_label[i]
   tag_scores = model3(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total6 += 1
   correct6 += (pred == targets).sum().item()
print('Training accuracy is :',correct6/total6*100)

total7 = 0
correct7 = 0
with torch.no_grad():
  for i in range(0,len(test)):
   inputs = Variable(test[i].view(1,25,4096))
   targets = test_label[i]
   tag_scores = model3(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total7 += 1
   correct7 += (pred == targets).sum().item()
print('Testing accuracy is :',correct7/total7*100)

Training accuracy is : 32.21253632212536
Testing accuracy is : 27.865404837013667


In [0]:
# epoch = 50
total8 = 0
correct8 = 0
with torch.no_grad():
  for i in range(0,len(train)):
   inputs = Variable(train[i].view(1,25,4096))
   targets = train_label[i]
   tag_scores = model4(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total8 += 1
   correct8 += (pred == targets).sum().item()
print('Training accuracy is :',correct8/total8*100)

total9 = 0
correct9 = 0
with torch.no_grad():
  for i in range(0,len(test)):
   inputs = Variable(test[i].view(1,25,4096))
   targets = test_label[i]
   tag_scores = model4(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total9 += 1
   correct9 += (pred == targets).sum().item()
print('Testing accuracy is :',correct9/total9*100)

Training accuracy is : 67.78746367787464
Testing accuracy is : 52.05047318611987


In [10]:
# epoch = 100
total10 = 0
correct10 = 0
with torch.no_grad():
  for i in range(0,len(train)):
   inputs = Variable(train[i].view(1,25,4096))
   targets = train_label[i]
   tag_scores = model5(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total10 += 1
   correct10 += (pred == targets).sum().item()
print('Training accuracy is :',correct10/total10*100)

total11 = 0
correct11 = 0
with torch.no_grad():
  for i in range(0,len(test)):
   inputs = Variable(test[i].view(1,25,4096))
   targets = test_label[i]
   tag_scores = model5(inputs)
   _, pred = torch.max(tag_scores.data, 1)
   total11 += 1
   correct11 += (pred == targets).sum().item()
print('Testing accuracy is :',correct11/total11*100)

Training accuracy is : 39.10336239103363
Testing accuracy is : 29.33753943217666


* ##### **Print the train and test and test accuracy of SVM** 

In [0]:
train_svc = []
train_svc_label = []
for i in range(0,len(train)):
  train_svc.append(train[i])
  for j in range(0,25):
    train_svc_label.append(train_label[i])
train_svc = torch.cat(train_svc).numpy()
train_svc_label = torch.cat(train_svc_label).numpy()

test_svc = []
test_svc_label = []
for i in range(0,len(test)):
  test_svc.append(test[i])
  for j in range(0,25):
    test_svc_label.append(test_label[i])
test_svc = torch.cat(test_svc).numpy()
test_svc_label = torch.cat(test_svc_label).numpy()

In [0]:
# Don't hardcode the train and test accuracy
from sklearn.svm import LinearSVC
model1 = LinearSVC()
model1.fit(train_svc, train_svc_label)
pred1 = model1.predict(test_svc)

In [0]:
correct2 = 0
total2 = len(pred1)
correct2 =+ (pred1 == test_svc_label).sum().item()
print('Testing accuracy is:',correct2/total2*100)

Testing accuracy is: 80.64353312302839


## **Problem 4.** Report

## **Bonus**


* ##### **Print the size of your training and test data**

In [0]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :', )
print('Shape of test/validation data is :', )

* ##### **Modelling and evaluation**

In [0]:
#Write your code for modelling and evaluation

## Submission
---
**Runnable source code in ipynb file and a pdf report are required**.

The report should be of 3 to 4 pages describing what you have done and learned in this homework and report performance of your model. If you have tried multiple methods, please compare your results. If you are using any external code, please cite it in your report. Note that this homework is designed to help you explore and get familiar with the techniques. The final grading will be largely based on your prediction accuracy and the different methods you tried (different architectures and parameters).

Please indicate clearly in your report what model you have tried, what techniques you applied to improve the performance and report their accuracies. The report should be concise and include the highlights of your efforts.
The naming convention for report is **Surname_Givenname_SBUID_report*.pdf**

When submitting your .zip file through blackboard, please
-- name your .zip file as **Surname_Givenname_SBUID_hw*.zip**.

This zip file should include:
```
Surname_Givenname_SBUID_hw*
        |---Surname_Givenname_SBUID_hw*.ipynb
        |---Surname_Givenname_SBUID_hw*.pdf
        |---Surname_Givenname_SBUID_report*.pdf
```

For instance, student Michael Jordan should submit a zip file named "Jordan_Michael_111134567_hw5.zip" for homework5 in this structure:
```
Jordan_Michael_111134567_hw5
        |---Jordan_Michael_111134567_hw5.ipynb
        |---Jordan_Michael_111134567_hw5.pdf
        |---Jordan_Michael_111134567_report*.pdf
```

The **Surname_Givenname_SBUID_hw*.pdf** should include a **google shared link**. To generate the **google shared link**, first create a folder named **Surname_Givenname_SBUID_hw*** in your Google Drive with your Stony Brook account. 

Then right click this folder, click ***Get shareable link***, in the People textfield, enter two TA's emails: ***bo.cao.1@stonybrook.edu*** and ***sayontan.ghosh@stonybrook.edu***. Make sure that TAs who have the link **can edit**, ***not just*** **can view**, and also **uncheck** the **Notify people** box.

Colab has a good feature of version control, you should take advantage of this to save your work properly. However, the timestamp of the submission made in blackboard is the only one that we consider for grading. To be more specific, we will only grade the version of your code right before the timestamp of the submission made in blackboard. 

You are encouraged to post and answer questions on Piazza. Based on the amount of email that we have received in past years, there might be dealys in replying to personal emails. Please ask questions on Piazza and send emails only for personal issues.

Be aware that your code will undergo plagiarism check both vertically and horizontally. Please do your own work.